In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder
from library_dicom.dicom_processor.model.FusionPET_CT import FusionPET_CT

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [ ]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [2]:
#LIST_PATH from json
json_path = '/media/oncopole/DD 2To/REMARC_Validated_DICOMS/REMARC_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [3]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

number of study :  202


In [4]:
#ADD CSV FILE 

In [ ]:
#add csv_file from csv 

csv_path= '/home/deeplearning/AHL/AHL_CSV_DATA/pet0.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [4]:
#add csv_file from a list 

csv_path = '/media/oncopole/DD 2To/REMARC_csv'
liste_csv_file = os.listdir(csv_path)
liste_csv = []
for csv_file in liste_csv_file : 
    liste_csv.append(os.path.join(csv_path, csv_file))



for data in dataset : 
    for csv_f in liste_csv : 
        if data[-1] in csv_f : 
            data.append(csv_f)

In [5]:
check_csv = []
for data in dataset : 
    if len(data) != 8 : 
        check_csv.append(data)

print(len(check_csv))
write_json_file('/media/oncopole/DD 2To/REMARC_Validated_DICOMS', 'check_csv', check_csv)

for r in check_csv : 
    dataset.remove(r)

20


#rewrite path 

for data in dataset : 
    new_1 = data[0].replace('DD 2To', '83c5223d-7a01-4ed0-b268-b877a7da96e2')
    new_2 = data[2].replace('DD 2To', '83c5223d-7a01-4ed0-b268-b877a7da96e2')
    data[0] = new_1
    data[2] = new_2

In [6]:
print('Nombre de study : ', len(dataset))

Nombre de study :  182


In [7]:
write_json_file('/media/oncopole/DD 2To/REMARC_Validated_DICOMS', 'validated_dataset', dataset)

In [ ]:
#remove remove_bi_file if it exists in every serie 
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

In [ ]:
#Check if there is Unconstant Spacing error in every serie of the dataset 
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [ ]:
#Careful : maybe need to rewrite path if the series are in an other folder

In [8]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)

nifti_directory = '/media/oncopole/DD 2To/REMARC_NIFTI'
mip_directory = '/media/oncopole/DD 2To/REMARC_MIP'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 

                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")

                #generation merged PT/CT
                filename_merged = study_uid+'_'+'nifti_'+'merged'+'.nii'
                fusion_objet = FusionPET_CT(serie_pt_objet, serie_ct_objet, target_size, target_spacing, target_direction)
                pet_img, ct_img = fusion_objet.generate_pet_ct_img()
                merged_img = fusion_objet.save_nifti_fusion(pet_img, ct_img, os.path.join(nifti_directory, filename_merged), mode ='head')
                print("EXPORT NIFTI MERGED")




            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print('MIP PET')
                #mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK')
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")

                #generation merged PT/CT
                filename_merged = study_uid+'_'+'nifti_'+'merged'+'.nii'
                fusion_objet = FusionPET_CT(serie_pt_objet, serie_ct_objet, target_size, target_spacing, target_direction)
                pet_img, ct_img = fusion_objet.generate_pet_ct_img()
                merged_img = fusion_objet.save_nifti_fusion(pet_img, ct_img, os.path.join(nifti_directory, filename_merged), mode ='head')
                print("EXPORT NIFTI MERGED")




            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print("MIP PET")
                #mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK ')
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

7
FALSE MASK
{1: {'SUV_max': 27.06, 'SUV_mean': 16.41, 'SD': 4.16}, 2: {'SUV_max': 25.35, 'SUV_mean': 15.83, 'SD': 4.0}, 3: {'SUV_max': 6.41, 'SUV_mean': 4.06, 'SD': 1.09}, 4: {'SUV_max': 3.88, 'SUV_mean': 3.03, 'SD': 0.31}, 5: {'SUV_max': 5.13, 'SUV_mean': 3.2, 'SD': 0.67}, 6: {'SUV_max': 6.11, 'SUV_mean': 3.56, 'SD': 1.02}, 7: {'SUV_max': 28.64, 'SUV_mean': 18.34, 'SD': 3.91}, 8: {'SUV_max': 22.74, 'SUV_mean': 13.96, 'SD': 3.06}, 9: {'SUV_max': 10.32, 'SUV_mean': 6.18, 'SD': 1.73}, 10: {'SUV_max': 18.85, 'SUV_mean': 11.2, 'SD': 2.86}, 11: {'SUV_max': 5.31, 'SUV_mean': 3.49, 'SD': 0.75}, 12: {'SUV_max': 21.28, 'SUV_mean': 13.47, 'SD': 3.39}, 13: {'SUV_max': 23.71, 'SUV_mean': 14.31, 'SD': 3.4}, 14: {'SUV_max': 26.68, 'SUV_mean': 17.75, 'SD': 4.34}, 15: {'SUV_max': 28.16, 'SUV_mean': 17.31, 'SD': 3.93}, 16: {'SUV_max': 8.1, 'SUV_mean': 4.85, 'SD': 1.29}, 17: {'SUV_max': 4.71, 'SUV_mean': 3.1, 'SD': 0.45}, 18: {'SUV_max': 25.67, 'SUV_mean': 17.2, 'SD': 3.62}, 19: {'SUV_max': 27.49, 'SUV

In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

In [ ]:
#Save different json 
directory_nifti = ''
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)

write_json_file(directory_nifti, 'error', error_dataset)

mip directory = directory_nifti+'/'+'MIP'
write_json_file(mip_directory, 'path_mip', path_mip)

In [ ]:
#create pdf of MIP for false mask 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#create csv 

In [ ]:
#Generate PET, CT, MASK NIFTI without checking

nifti_directory = nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

In [ ]:
#Generate PET & CT NIFTI only

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI/pet4'

error = []
for serie in pet_4 : 
    print(pet_4.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

In [ ]:
#Generate PET only

nifti_directory = '/home/deeplearning/AHL/suv_test'
dataset = dataset[0:10]

error = []
for serie in dataset : 
    print(dataset.index(serie))
    print(serie[0])
    try : 
                serie_pt_objet = SeriesPT(serie[1]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)